## EZKL Jupyter Notebook Demo

Here we demonstrate the use of the EZKL package in a Jupyter notebook whereby all components of the circuit are public or pre-committed to. This is the simplest case of using EZKL (proof of computation).

In [ ]:
# check if notebook is in colab
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass


# here we create and (potentially train a model)

# make sure you have the dependencies required here already installed
from torch import nn
import ezkl
import os
import json
import torch


# Defines the model
# we got convs, we got relu, we got linear layers
# What else could one want ????

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=5, stride=2)
        self.conv2 = nn.Conv2d(in_channels=2, out_channels=3, kernel_size=5, stride=2)

        self.relu = nn.ReLU()

        self.d1 = nn.Linear(48, 48)
        self.d2 = nn.Linear(48, 10)

    def forward(self, x):
        # 32x1x28x28 => 32x32x26x26
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)

        # flatten => 32 x (32*26*26)
        x = x.flatten(start_dim = 1)

        # 32 x (32*26*26) => 32x128
        x = self.d1(x)
        x = self.relu(x)

        # logits => 32x10
        logits = self.d2(x)

        return logits


circuit = MyModel()

# Train the model as you like here (skipped for brevity)




In [ ]:
model_path = os.path.join('network.onnx')
compiled_model_path = os.path.join('network.compiled')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')

witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')

In [ ]:

shape = [1, 28, 28]
# After training, export to onnx (network.onnx) and create a data file (input.json)
x = 0.1*torch.rand(1,*shape, requires_grad=True)

# Flips the neural net into inference mode
circuit.eval()

    # Export the model
torch.onnx.export(circuit,               # model being run
                      x,                   # model input (or a tuple for multiple inputs)
                      model_path,            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=10,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})

data_array = ((x).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

    # Serialize data into file:
json.dump( data, open(data_path, 'w' ))


In [ ]:

py_run_args = ezkl.PyRunArgs()
py_run_args.input_visibility = "public"
py_run_args.output_visibility = "public"
py_run_args.param_visibility = "fixed" # "fixed" for params means that the committed to params are used for all proofs

res = ezkl.gen_settings(model_path, settings_path, py_run_args=py_run_args)
assert res == True


In [ ]:
cal_path = os.path.join("calibration.json")

data_array = (torch.rand(20, *shape, requires_grad=True).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

# Serialize data into file:
json.dump(data, open(cal_path, 'w'))


await ezkl.calibrate_settings(cal_path, model_path, settings_path, "resources")

In [ ]:
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

In [ ]:
# srs path
res = ezkl.get_srs( settings_path)

In [ ]:
# now generate the witness file

res = await ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

In [ ]:

# HERE WE SETUP THE CIRCUIT PARAMS
# WE GOT KEYS
# WE GOT CIRCUIT PARAMETERS
# EVERYTHING ANYONE HAS EVER NEEDED FOR ZK



res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,

    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

In [ ]:
# GENERATE A PROOF


proof_path = os.path.join('test.pf')

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,

        "single",
    )

print(res)
assert os.path.isfile(proof_path)

In [ ]:
# VERIFY IT

res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,

    )

assert res == True
print("verified")

In [ ]:
!pip install ezkl onnx
import torch
import torch.nn as nn
import torch.onnx
import json
import os
import ezkl
import onnx
import nest_asyncio
nest_asyncio.apply()


In [ ]:
# STEP 0: Install dependencies
!pip install ezkl sentence-transformers scikit-learn nest_asyncio

import torch
import torch.nn as nn
import torch.onnx
import json
import os
import ezkl
import numpy as np
from sklearn.decomposition import PCA
from sentence_transformers import SentenceTransformer
import nest_asyncio
nest_asyncio.apply()


In [ ]:
# STEP 1: Define and export the PyTorch model (input size = 32)
class SimpleZKMLModel(nn.Module):
    def __init__(self):
        super(SimpleZKMLModel, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.net(x)

model = SimpleZKMLModel()
model.eval()

# Save ONNX model
onnx_model_path = "zk_model.onnx"
dummy_input = torch.rand(1, 32)
torch.onnx.export(
    model, dummy_input, onnx_model_path,
    input_names=["input"], output_names=["output"],
    opset_version=11
)


In [ ]:
# STEP 2: Prepare real sentence embeddings and PCA projection to fit input
encoder = SentenceTransformer("sentence-transformers/paraphrase-MiniLM-L6-v2")

# Train PCA (need at least 16+ samples)
samples = [
    "Cats are animals.", "Dogs are loyal.", "The sky is blue.", "Water boils at 100C.",
    "Sun rises in the east.", "Fire is hot.", "Snow is cold.", "Fish swim.",
    "Birds fly.", "Humans need oxygen.", "The Earth orbits the Sun.", "Time is continuous.",
    "Energy is conserved.", "Gravity pulls downward.", "Sound travels in waves.",
    "Light is electromagnetic.", "Plants need sunlight.", "Electrons carry charge."
]
X = encoder.encode(samples)
pca = PCA(n_components=16)
pca.fit(X)

def preprocess(claim, evidence):
    c_vec = encoder.encode(claim)
    e_vec = encoder.encode(evidence)
    c_pca = pca.transform([c_vec])[0]
    e_pca = pca.transform([e_vec])[0]
    return np.concatenate([c_pca, e_pca]).tolist()

# Example sentence pair
vec = preprocess("The Earth is flat", "NASA shows Earth is round")
input_data = { "input_data": [vec] }

with open("input.json", "w") as f:
    json.dump(input_data, f)


In [ ]:
# STEP 3: Generate settings and calibrate
settings_path = "settings.json"
py_run_args = ezkl.PyRunArgs()
py_run_args.input_visibility = "private"
py_run_args.output_visibility = "public"
py_run_args.param_visibility = "private"
py_run_args.logrows = 17

assert ezkl.gen_settings(onnx_model_path, settings_path, py_run_args)
assert await ezkl.calibrate_settings("input.json", onnx_model_path, settings_path, "resources")


In [ ]:
# STEP 4: Compile circuit
compiled_model_path = "zk_model.ezkl"
assert ezkl.compile_circuit(onnx_model_path, compiled_model_path, settings_path)


In [ ]:
# STEP 5: Generate witness for the claim-evidence pair
witness_path = "witness.json"
assert await ezkl.gen_witness("input.json", compiled_model_path, witness_path)


In [ ]:
# !rm -f ~/.ezkl/srs/kzg15.srs
# !mkdir -p ~/.ezkl/srs
# !curl -L -o ~/.ezkl/srs/kzg15.srs https://ezkl-public.s3.us-west-2.amazonaws.com/kzg15.srs


In [ ]:
# from google.colab import files

# uploaded = files.upload()  # Upload your local kzg15.srs file


In [ ]:
# !mkdir -p ~/.ezkl/srs
# !mv kzg15.srs ~/.ezkl/srs/kzg15.srs



In [ ]:
# !ls -lh ~/.ezkl/srs/kzg15.srs

In [ ]:
vk_path = "test.vk"
pk_path = "test.pk"
assert ezkl.setup(compiled_model_path, vk_path, pk_path)


In [ ]:
# STEP 7: Generate and verify proof
proof_path = "test.pf"
assert ezkl.prove(witness_path, compiled_model_path, pk_path, proof_path, "single")
assert ezkl.verify(proof_path, settings_path, vk_path)
print("✅ Proof verified!")


In [ ]:
# Swap in new claim + evidence
new_vec = preprocess("Apples are fruit", "Apples grow on trees")
with open("input.json", "w") as f:
    json.dump({ "input_data": [new_vec] }, f)

# Re-run witness → prove → verify
assert await ezkl.gen_witness("input.json", compiled_model_path, witness_path)
assert ezkl.prove(witness_path, compiled_model_path, pk_path, proof_path, "single")
assert ezkl.verify(proof_path, settings_path, vk_path)
print("✅ New claim-evidence pair verified!")


In [ ]:
class SimpleZKMLModel(nn.Module):
    def __init__(self):
        super(SimpleZKMLModel, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.net(x)

model = SimpleZKMLModel()
model.eval()


In [ ]:
x = torch.rand(1, 32)
onnx_model_path = "zk_model.onnx"

torch.onnx.export(
    model,
    x,
    onnx_model_path,
    input_names=["input"],
    output_names=["output"],
    dynamic_axes=None,
    opset_version=11
)


In [ ]:
input_data = {
    "input_data": [x.detach().numpy().reshape(-1).tolist()]
}

with open("input.json", "w") as f:
    json.dump(input_data, f)


In [ ]:
settings_path = "settings.json"
py_run_args = ezkl.PyRunArgs()
py_run_args.input_visibility = "private"
py_run_args.output_visibility = "public"
py_run_args.param_visibility = "private"
py_run_args.logrows = 17  # Adjustable if needed

res = ezkl.gen_settings(onnx_model_path, settings_path, py_run_args=py_run_args)
assert res == True


In [ ]:
res = await ezkl.calibrate_settings("input.json", onnx_model_path, settings_path, "resources")
assert res == True


In [ ]:
compiled_model_path = "zk_model.ezkl"
res = ezkl.compile_circuit(onnx_model_path, compiled_model_path, settings_path)
assert res == True


In [ ]:
witness_path = "witness.json"
res = await ezkl.gen_witness("input.json", compiled_model_path, witness_path)
assert os.path.isfile(witness_path)


In [ ]:
vk_path = "test.vk"
pk_path = "test.pk"

res = ezkl.setup(compiled_model_path, vk_path, pk_path)
assert res == True


In [ ]:
proof_path = "test.pf"

res = ezkl.prove(witness_path, compiled_model_path, pk_path, proof_path, "single")
assert os.path.isfile(proof_path)


In [ ]:
res = ezkl.verify(proof_path, settings_path, vk_path)
assert res == True
print("✅ Proof Verified!")


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
import numpy as np
import json

encoder = SentenceTransformer("sentence-transformers/paraphrase-MiniLM-L6-v2")

# Collect >16 samples to train PCA
samples = [
    "Cats are animals.",
    "Dogs are loyal.",
    "The sky is blue.",
    "Water boils at 100C.",
    "Sun rises in the east.",
    "Fire is hot.",
    "Snow is cold.",
    "Fish swim.",
    "Birds fly.",
    "Humans need oxygen.",
    "The Earth orbits the Sun.",
    "Time is continuous.",
    "Energy is conserved.",
    "Gravity pulls downward.",
    "Sound travels in waves.",
    "Light is electromagnetic.",
    "Plants need sunlight.",
    "Electrons carry charge."
]

X = encoder.encode(samples)
pca = PCA(n_components=16)
pca.fit(X)

def preprocess(claim, evidence):
    c_vec = encoder.encode(claim)
    e_vec = encoder.encode(evidence)
    c_pca = pca.transform([c_vec])[0]
    e_pca = pca.transform([e_vec])[0]
    final_input = np.concatenate([c_pca, e_pca])
    return final_input.tolist()

# Your claim and evidence
input_vec = preprocess("The Earth is flat", "NASA shows Earth is round")

with open("input.json", "w") as f:
    json.dump({ "input_data": [input_vec] }, f)


In [ ]:
await ezkl.gen_witness("input.json", "zk_model.ezkl", "witness.json")
ezkl.prove("witness.json", "zk_model.ezkl", "test.pk", "test.pf", "single")
ezkl.verify("test.pf", "settings.json", "test.vk")


In [ ]:
res = await ezkl.gen_witness("input.json", "zk_model.ezkl", "witness.json")
assert res

res = ezkl.prove("witness.json", "zk_model.ezkl", "test.pk", "test.pf", "single")
assert res

res = ezkl.verify("test.pf", "settings.json", "test.vk")
assert res
print("✅ New claim-evidence pair verified!")
